In [14]:
import pandas as pd


In [25]:

import openai
from credentials import gpt_key
import json
from textwrap import dedent

In [26]:
client = openai.Client(api_key=gpt_key)

# Load Labels

In [27]:
labels = pd.read_csv("subtask2_all_fine-grained_narratives.csv")

In [28]:
labels[["top", "mid", "low"]] = labels["narrative"].str.split(": ",expand=True)

In [29]:
top_labels = labels["top"].unique().tolist()
top_labels

['CC', 'URW', 'Other']

In [30]:
mid_labels = labels["mid"].unique().tolist()
mid_labels

['Amplifying Climate Fears',
 'Climate change is beneficial',
 'Controversy about green technologies',
 'Criticism of climate movement',
 'Criticism of climate policies',
 'Criticism of institutions and authorities',
 'Downplaying climate change',
 'Green policies are geopolitical instruments',
 'Hidden plots by secret schemes of powerful groups',
 'Questioning the measurements and science',
 'Amplifying war-related fears',
 'Blaming the war on others rather than the invader',
 'Discrediting the West, Diplomacy',
 'Discrediting Ukraine',
 'Distrust towards Media',
 'Negative Consequences for the West',
 'Overpraising the West',
 'Praise of Russia',
 'Russia is the Victim',
 'Speculating war outcomes',
 None]

In [31]:
urw_labels = labels[labels["top"] == "URW"]["mid"].unique().tolist()
cc_labels = labels[labels["top"] == "CC"]["mid"].unique().tolist()

In [32]:
top_classifier_instruction = """'You are a classifier that classifies input into three categories: "URW", "CC", and "Other".
URW refers to the Ukraine-Russia War, CC refers to Climate Change, and Other refers to all other topics.
Follow a strict json format for the output. The JSON format is as follows:\n              {"top_class": str}
    The top_class key should have one of the three values: "URW", "CC", or "Other".
Sentences talking about Ukraine or Russia which are not related their war should be classified as "Other".
talking about climate but not climate change should be classified as "Other"."""
print(top_classifier_instruction)

'You are a classifier that classifies input into three categories: "URW", "CC", and "Other".
URW refers to the Ukraine-Russia War, CC refers to Climate Change, and Other refers to all other topics.
Follow a strict json format for the output. The JSON format is as follows:
              {"top_class": str}
    The top_class key should have one of the three values: "URW", "CC", or "Other".
Sentences talking about Ukraine or Russia which are not related their war should be classified as "Other".
talking about climate but not climate change should be classified as "Other".


In [33]:
def call_agent(instruction, prompt):
    response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system", 
            "content": dedent(instruction)
        },
        {
            "role": "user", 
            "content": prompt
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "class",
            "schema": {
                "type": "object",
                "properties": {
                    "class": {"type": "string"},
                },
                "required": ["class"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
    )
    return json.loads(response.choices[0].message.content)

In [77]:



prompt = "NASA's James Hansen claims extreme weather events would not have happened if CO2 had remained at its pre-industrial level of 280 parts per million"
prompt = "Nuccitellis answer to this unexceptionable and temperately-expressed statement is that sea level rises chiefly through thermal expansion and melting land ice, so it is all our fault. However, it ought to have been obvious even to Nuccitelli thermal expansion and land-ice melt happen whether Man or nature is the cause."

prompt = """People concerned that manmade greenhouse gas emissions, in particular carbon dioxide (CO2), may be causing dangerous climate change worry such a change may disrupt the hydrosphere, which comprises all of the water on Earth and in its atmosphere. This disruption, they argue, would lead to increased rainfall and erratic weather patterns that would greatly increase sea level and produce potentially devastating consequences.A report from the Nongovernmental International Panel on Climate Change (NIPCC), an independent group of some 50 scientists from 15 countries, titled Climate Change Reconsidered II: Physical Science, summarizes a large body of research disputing these claims and finds the hydrosphere has exhibited only mild volatility over the past century in concert with natural climate cycles and with no correlation to human CO2 emissions.The data show short-term precipitation volatility is not correlated with either CO2 emissions or general climate warming. For example, monsoon intensity displayed no increase later in the century despite increased CO2 emissions, but instead was correlated with solar activity.In addition, monsoons were generally more intensive during the Little Ice Age (LIA) than the Medieval Warm Period (MWP), which runs directly contrary to the alarmist claims. On the other end of the volatility spectrum, CCR-II disputes the strength of the commonly cited connection between global warming and droughts, as evidenced by the similar rates of drought occurrence in the LIA and MWP.As with precipitation, CCR-II finds no significant connection between sea level or ocean temperature and human CO2 emissions. Sea level is determined by a multitude of factors, and CO2-induced global warming is a minor one. Ocean temperatures have remained virtually unchanged for nine years, according to data of the Argo buoy network.CCR-II found numerous problems with the consensus view of sea-level change. The global sea level has increased by an average of only 12 mm annually over the century, with a wide range of intra-year volatility between +5mm and -5mm away from the mean. Supposed instances of environmental harm caused by sea-level rises, such as coral reef destruction, are often the result of unaccounted alternative factors.Concern over manmade greenhouse gas emissions has been the impetus for many destructive public policies, including renewable portfolio standards, alternative energy subsidies, and high gasoline taxes. Yet, despite ever-increasing levels of CO2 emissions, scientists have been unable to demonstrate a connection to its alleged effects, such as hydrosphere alteration, over the previous century. CCR-II's findings demonstrate precipitation and ocean activity have little to no connection to human CO2 emissions. Therefore, energy production, the lifeblood of the global economy, should not be hindered by state intervention to reduce CO2 emissions.The above introduction was based on text from Climate Change Reconsidered II: Physical Science and its Summary for Policymakers, published by the Nongovernmental International Panel on Climate Change (NIPCC).The following documents provide additional information about the hydrosphere and oceans. Chapter 6 of Climate Change Reconsidered IIhttp://heartland.org/media-library/pdfs/CCR-II/Chapter-6-Hydrosphere-Oceans.pdfIn Chapter Six of Climate Change Reconsidered II: Physical Science, Willem de Lange and Robert M. Carter find little evidence for an overall increase in global precipitation during the twentieth century independent of natural multidecadal climate rhythms. Summary for Policymakers of Climate Change Reconsidered IIhttp://heartland.org/media-library/pdfs/CCR-II/Summary-for-Policymakers.pdfThe IPCC claims to know, apparently with rising certainty over time, that "most of the observed increase in global average temperatures since the mid-20th century is very likely due to the observed increase in anthropogenic greenhouse gas concentrations" (IPCC AR4 SPM, p. 10). This Summary for Policymakers summarizes and interprets Climate Change Reconsidered II, a major scientific report that refutes this claim. More IPCC Misdirection: Its Dodgy Sea Level-Rise Assessmenthttp://www.cato.org/blog/more-ipcc-misdirection-its-dodgy-sea-level-rise-assessmentPatrick J. Michaels and Paul C. Knappenberger of the Cato Institute criticize the Intergovernmental Panel on Climate Change's analysis of sea-level changes in a September 24, 2013 blog post. The authors point out the IPCC cherry-picks data clumps that coincide with their conclusions, while ignoring broader data that contradict them. For instance, the IPCC attributes the recent abnormally high increases in sea level to a new long-term trend, even though the rate of increase has already begun to decline to normal levels, thereby indicating this period was just another short-term cyclical increase. Precipitation Measurements and Trends in the Twentieth Century http://heartland.org/policy-documents/precipitation-measurements-and-trends-twentieth-centuryA research paper published in the International Journal of Climatology in 2002 presents the trends in precipitation over the last century. Global precipitation increased by only 9 mm per year over the century, despite enormous short-term volatility of up to 40mm. Short-term variability can be attributed to natural climate cycles, including reoccurring ENSOs, Arctic Oscillation, and Antarctic Oscillation. The authors admit global precipitation measurement is still primitive and not wholly reliable. Little Change in Global Drought Over the Past 60 Yearshttp://heartland.org/policy-documents/little-change-global-drought-over-past-60-yearsIn this paper published in 2012 in Nature, Justin Sheffield, Eric F. Wood, and Michael L. Roderick cast doubt on the supposed connection between global warming and the frequency of droughts over the past 60 years. The standard analysis used to estimate global moisture levels is highly flawed and underestimates moisture levels, they note. In reality, global moisture levels have barely changed, and by extension, theories that blame hydrosphere changes on global warming are also flawed. Research Explores Hidden Benefits of Intensive Rainfall in East Africahttp://phys.org/news/2012-11-explores-hidden-benefits-intensive-rainfall.htmlMaggie Clune looks at a new study published in 2012 in Nature Climate Change, which demonstrates some benefits of increased rainfall. Generally, climate scientists tend to focus on only the negative effects of climate change while ignoring the positive results it produces. One of these is that increased rainfall replenishes freshwater sources more often. This makes underwater aquifers, rivers, streams, etc. more bountiful, especially in technologically less-developed regions that are heavily dependent upon natural water sources, such as Central Africa. Indian Monsoon Variability in a Global Warming Scenariohttp://heartland.org/policy-documents/indian-monsoon-variability-global-warming-scenarioA paper published in 2003 in Natural Hazards applies to real historical data the IPCC's theory that global warming increases monsoon intensity. The researchers determined there is no correlation between monsoon intensity and global warming. Monsoon strength naturally oscillates along decade-long time intervals regardless of carbon production. Furthermore, increased global temperatures have no effect on intra-cycle variability. Nothing in this Research & Commentary is intended to influence the passage of legislation, and it does not necessarily represent the views of The Heartland Institute. For further information on this and other topics, visit the Environment & Climate News Web site at http://news.heartland.org/energy-and-environment, The Heartland Institute's Web site at http://www.heartland.org, and PolicyBot, Heartland's free online research database, at www.policybot.org.If you have any questions about this issue or The Heartland Institute, or would like a scientist to testify in your state, contact Heartland Institute Policy Analyst Taylor Smith at tsmith@heartland.org or 312/377-4000"""
# prompt = """The discussion is far from over. The true mind of science remains open to new data and alternative explanations. Whether and how much of the approximately 1/2 degree C of warming which occurred in the latter half of the 20th century is due to human industry has not been conclusively established. Neither have any of the incredibly expensive "solutions" proposed to address any global warming been shown to be meaningfully effective or worth their tremendous cost."""


def classifier(row):
    prompt = row["text"]
    file = row["file"]


    top_classifier_output = {"class": None}
    urw_classifier_output = {"class": None}
    cc_classifier_output = {"class": None}

    fine_grained_output = {"class": None}
    top_classifier_output = call_agent(top_classifier_instruction, prompt)

    if top_classifier_output["class"] == "Other":
        return file, "Other", "Other"
    elif top_classifier_output["class"] == "URW":
        urw_classifier_instruction = f"""You are a disinformation classifier regarding the Ukraine-Russia War. 
        You have to classify the input into {len(urw_labels)} categories. The categories are: {urw_labels}.
        Your output should be strict json format. The JSON format is as follows:
        {{"class": str}}.
        If you can't classify the input into any of the categories, classify it as "Other"."""
        urw_classifier_output = call_agent(urw_classifier_instruction, prompt)

        fine_grain_urw_labels = labels[labels["mid"] == urw_classifier_output["class"]]["low"].unique().tolist()

        fine_grained_instruction = f"""You are a disinformation classifier that classifies input into {len(fine_grain_urw_labels)} categories from {urw_classifier_output["class"]} narrative.
        The categories are:{fine_grain_urw_labels}.
        Your output should be strict json format. The JSON format is as follows:
        {{"class": str}}.
        If you can't classify the input into any of the categories, classify it as "Other".
        """
        fine_grained_output = call_agent(fine_grained_instruction, prompt)
        
        # result =  top_classifier_output["class"], ": ", urw_classifier_output["class"],": ", fine_grained_output["class"]
        narrative = top_classifier_output["class"] + ": " + urw_classifier_output["class"]
        subnarrative = narrative + ": " + fine_grained_output["class"]
        return file, narrative, subnarrative





    elif top_classifier_output["class"] == "CC":
        cc_classifier_instruction = f"""You are a disinformation classifier regarding Climate Change.
        You have to classify the input into {len(cc_labels)} categories. The categories are: {cc_labels}.
        Your output should be strict json format. The JSON format is as follows:
        {{"class": str}}.
        If you can't classify the input into any of the categories, classify it as "Other"."""
        cc_classifier_output = call_agent(cc_classifier_instruction, prompt)

        fine_grained_cc_labels = labels[labels["mid"] == cc_classifier_output["class"]]["low"].unique().tolist()
        fine_grained_instruction = f"""You are a disinformation classifier that classifies input into {len(fine_grained_cc_labels)} categories from {cc_classifier_output["class"]} narrative.
        The categories are:{fine_grained_cc_labels}.
        Your output should be strict json format. The JSON format is as follows:
        {{"class": str}}.
        If you can't classify the input into any of the categories, classify it as "Other".
        """
        fine_grained_output = call_agent(fine_grained_instruction, prompt)
        result =  top_classifier_output["class"], ": ", cc_classifier_output["class"],": ", fine_grained_output["class"]\
        # narrative = ": ".join(top_classifier_output["class"], cc_classifier_output["class"])
        # subnarrative = ": ".join(narrative, fine_grained_output["class"])
        narrative = top_classifier_output["class"] + ": " + cc_classifier_output["class"]
        subnarrative = narrative + ": " + fine_grained_output["class"]
        return file, narrative, subnarrative


# load devset

In [63]:
devset = pd.read_csv("dev.csv")

In [64]:
devset = devset[["file", "text"]]

In [75]:
res = classifier(devset.iloc[25])

In [76]:
res

('EN_UA_DEV_100003.txt',
 'URW: Praise of Russia: Praise of Russian military might')

In [60]:
devset

,file,text
0,EN_CC_200030.txt,"Bangladesh, Nordic countries to strengthen coo..."
1,EN_CC_200033.txt,Greta Thunberg Calls For 'Overthrow of Whole C...
2,EN_CC_200034.txt,If we “just stop oil” like climate protesters ...
3,EN_CC_200035.txt,Gretchen Whitmer Orders Michigan’s State Fleet...
4,EN_CC_200036.txt,Climate cultists push bizarre scare language a...
5,EN_CC_200040.txt,Climate Protesters Out Of Control As They Atta...
6,EN_CC_200046.txt,CO2 is the GREENING molecule: New study shows ...
7,EN_CC_200047.txt,89% of ‘American Elites’ Back WEF’s Plan to Ra...
8,EN_CC_200049.txt,Alarmists Warn of U.S. ‘Heat Dome’ Tied to Hum...
9,EN_CC_200050.txt,Link to Major Banks Bend The Knee To Climate A...


In [78]:
all_res = []

for index, row in devset.iterrows():
    res = classifier(row)
    all_res.append(res)

In [80]:
res_df = pd.DataFrame(all_res)

In [83]:
res_df.to_csv("dev_results.txt", index=False, header=False, sep="\t")